In [9]:
!pip install pyserial

In [53]:
import serial

def read_data(ser):
    while True:
        if ser.in_waiting >= 7:
            data = ser.read(7)
            identifier = (data[0] >> 5) & 0x03
            analog_value = ((data[0] & 0x1F) << 5) | ((data[1] >> 3) & 0x1F)
            time_micros = (data[2] << 24) | (data[3] << 16) | (data[4] << 8) | data[5]
            
            print(f"Identifier: {identifier}, Analog Value: {analog_value}, Time: {time_micros}")

if __name__ == "__main__":
    ser = serial.Serial('/dev/cu.usbserial-110', 9600)  # Adjust the port as needed
    read_data(ser)


127
248
Identifier: 3, Analog Value: 1023, Time: 127531090
